In [60]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataset_prepare import ToStrLower, DropDuplicates, FilterByQuantile
from autocompleter import AutoCompleterBaseline, AutoCompleteBySuffix

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
popular_queries = pd.read_csv("data/query_popularity.csv")
popular_queries['query'] = popular_queries['query'].astype(str).str.lower()

In [4]:
columns = ['wbuser_id', 'UQ', 'cnt', 'locale', 'weekday', 'time']
search_history = pd.read_csv("data/search_subset.csv")[columns]

# Filter data

In [5]:
CNT_UPPER_QUANTILE = 0.9

In [6]:
# to lower UQ
to_str_lower_filter = ToStrLower(column="UQ")
df = to_str_lower_filter.apply(search_history)

In [7]:
# filter out duplicates by wbuser_id 
subset_drop = ["wbuser_id", "UQ"]
drop_duplicates_filter = DropDuplicates(subset_drop)
df = drop_duplicates_filter.apply(df)

DropDuplicates: dataset size 499999 -> 450085


In [8]:
UPPER_QUANTILE = 0.95
LOWER_QUANTILE = 0
quantile_filter = FilterByQuantile('cnt', UPPER_QUANTILE)
df = quantile_filter.apply(df)

Lower quantile: 0. Upper quantile: 33128.79999999999
FilterByQuantile: dataset size 450085 -> 427580


### Autocompleters

#### Baseline

In [70]:
# baseline
autocompleter_baseline = AutoCompleterBaseline(query_database=popular_queries, max_n=10)
autocompleter_baseline.predict("игровой")

,query,query_popularity
652,игровой ноутбук,10
694,игровой ноут,10
5030,игровой компьютер,10
6583,игровой руль,10
8203,игровой комплекс для кошек,10
15001,игровой компьютерный стол,10
23432,игровой комплекс для кошки,10
30519,игровой столик,10
33981,игровой центр батут,9
38301,игровой набор кухня детский,9


In [73]:
autocompleter_baseline.predict("игровой ноут")

Found for word: игровой 169 instances
Found for word: ноут 411 instances


,query,query_popularity
268,ноутбук игровой,10
652,игровой ноутбук,10
694,игровой ноут,10


#### Tries

In [74]:
autocompleter_tries = AutoCompleteBySuffix()
autocompleter_tries.make_trie(popular_queries)

Making database for queries... It length: 336988


In [76]:
autocompleter_tries.query("игровой")[:10]

[('игровой ноут', 1, 10),
 ('игровой ноутбук', 1, 10),
 ('игровой компьютер', 1, 10),
 ('игровой компьютерный стол', 1, 10),
 ('игровой комплекс для кошек', 1, 10),
 ('игровой комплекс для кошки', 1, 10),
 ('игровой руль', 1, 10),
 ('игровой столик', 1, 10),
 ('игровой набор', 2, 9),
 ('игровой набор кухня детский', 1, 9)]

In [77]:
autocompleter_tries.query("игровая")[:10]

[('игровая приставка', 1, 10),
 ('игровая палатка', 1, 10),
 ('игровая мышь', 2, 10),
 ('игровая клавиатура', 2, 10),
 ('игровая приставка sega', 1, 9),
 ('игровая консоль', 1, 9),
 ('игровая гарнитура', 1, 9),
 ('игровая гарнитура rush ambition', 1, 9),
 ('игровая пристака', 1, 8),
 ('игровая мышь zet', 1, 8)]